# Text 생성 및 Prompt 예제

### Text 생성 API 비교
| 특징                                                                                            | **Responses API**                                                         | **Chat Completions API**                                              |
| --------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------- | --------------------------------------------------------------------- |
| **상태 유지(Stateful)**                                                                           | ✅ 서버 측에서 **대화 상태를 관리(저장 및 이어쓰기 가능)**                                      |                                                                                                                |                                                                       |
| **내장 도구 지원(Built-in Tools)**                                                                  | ✅ `web_search`, `file_search`, `computer_use_preview` 등 자동 호출 가능          | ❌ 기본 제공 도구 없음. **Function calling은 가능하지만, 호출/실행은 직접 구현 필요**           |
| **구조 & 사용 방식**                                                                                | `instructions`, `input`, `tools`, `store`, `previous_response_id` 기반 API  | `messages=[{role, content}, ...]`, `temperature`, `max_tokens`, 등의 구조 |
| **응답 형식 (Output)**                                                                            | `response.output_text`, `response.output` 구조로 단일 메세지 응답 및 도구 호출 결과 포함     | `choices[0].message.content` 형태로 단일 응답만 반환                            |
| **사용 목적**                                                                                     | ✅ **에이전트/도구 사용, 파일/웹 검색, 상태 관리**가 필요한 복잡한 워크플로우에 적합                       | ✅ **챗봇, 단순 대화, instruction-following**에 적합                            |
| **미래 확장성**                                                                                    | 🔧 OpenAI Roadmap 중심 – Responses API가 Chat API의 기능을 통합 발전시키는 방향           | 📦 **기존 표준 API**, 계속 지원되며 업계 표준 유지                                    |

[1]: https://simonwillison.net/2025/Mar/11/responses-vs-chat-completions/?utm_source=chatgpt.com "OpenAI API: Responses vs. Chat Completions"
[2]: https://cookbook.openai.com/examples/responses_api/responses_example?utm_source=chatgpt.com "Web Search and States with Responses API - OpenAI Cookbook"


In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4.1-nano"

## Responses API

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [4]:
# Responses API 호출 예시
response = client.responses.create(
    model="gpt-4o",  # 또는 "gpt-4.1" 등 최신 모델
    instructions="당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.",
    input="안갔어 시장에 그녀는."
)

# 응답 결과 출력
print(response.output_text)

그녀는 시장에 가지 않았어.


## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

In [9]:
response = client.responses.create(
    model=Model,
    instructions="구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.",
    input="""
      최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 
      회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 
      네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 
      이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
      이 데이터를 과일명, 색상, 맛으로 분석해 주세요.
      """
)

In [10]:
print(response.output_text)

아래는 제공된 데이터를 과일명, 색상, 맛으로 분석하여 정리한 CSV 형식입니다:

```csv
과일명,색상,맛
구크럭스,보라색,사탕맛
로헤클(Loheckles),회색빛 파란색,시큼함(레몬 유사)
포유닛,밝은 녹색,풍미가 좋음(단맛보다)
루프노바,네온 핑크색,솜사탕 같은 맛
글로울(Glowls),옅은 오렌지색,신맛과 쓴맛 및 부식성
```

이 표는 각 과일의 이름, 색상, 그리고 맛을 명확하게 정리한 것입니다.


## 대화 상태를 수동으로 관리

| 역할       | 설명                                                                                      |
|------------|-------------------------------------------------------------------------------------------|
| developer  | 애플리케이션 개발자가 제공한 지침. **user 메시지보다 우선적으로 처리**됩니다.              |
| user       | 최종 사용자가 입력한 지침. developer 메시지보다 **우선순위가 낮습니다.**                  |
| assistant  | 모델이 생성한 응답 메시지. 응답 결과는 이 역할로 반환됩니다.                              |


In [42]:
response = client.responses.create(
    model=Model,
    input=[
        {"role": "user", "content": "안녕하세요."},
        {"role": "assistant", "content": "안녕하세요. 무었을 도와드릴까요?"},
        {"role": "user", "content": "딥러닝과 머신러닝의 차이점이 무었인가요?"},
    ],
)

print(response.output_text)

딥러닝과 머신러닝은 모두 인공지능(AI)의 하위 분야이지만, 서로 다른 개념입니다. 차이점을 간단히 설명드리자면:

**머신러닝 (Machine Learning):**
- 인공지능의 한 분야로, 명시적으로 프로그래밍하지 않고 데이터로부터 패턴이나 규칙을 학습하는 기술입니다.
- 다양한 알고리즘(예: 결정트리, 서포트 벡터 머신, 랜덤 포레스트 등)을 사용하여 데이터를 분석하고 예측이나 분류를 수행합니다.
- 일반적으로 특징(Feature)을 수작업으로 만들어내는 과정이 필요할 수 있습니다.

**딥러닝 (Deep Learning):**
- 머신러닝의 하위 분야로, 인공신경망(특히 깊은 신경망, 딥 뉴럴 네트워크)을 활용합니다.
- 대량의 데이터와 많은 연산을 통해 자동으로 특징을 추출하고 학습하는 기술입니다.
- 이미지 인식, 음성 인식, 자연어 처리 등 복잡한 패턴을 다루는 데 강력합니다.

### 요약
| 항목             | 머신러닝                                          | 딥러닝                                              |
|------------------|------------------------------------------------|------------------------------------------------------|
| 정의             | 데이터로부터 규칙을 학습하는 일반적 방법         | 인공신경망을 이용한 심층 학습 방법                  |
| 특징 추출        | 수작업 또는 간단한 자동화                            | 자동으로 특징을 추출                                   |
| 데이터 필요량   | 상대적으로 적은 데이터에서도 가능                   | 대량의 데이터 필요                                  |
|

## Keyword 추출 

In [11]:
text = """
"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다. 
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다. 
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다. 
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다. 
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다. 
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."
"""

In [13]:
response = client.responses.create(
    model=Model,
    instructions="텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다.",
    input=text
)

In [14]:
print(response.output_text)

- 블랙 온 블랙 도자기 (Black-on-Black ware)
- 뉴멕시코 북부
- 푸에블로 원주민 도자기 예술가
- 20세기 및 21세기 도자기 전통
- 전통적인 환원 소성 블랙웨어
- 표면 매끄러움
- 선택적 버니싱
- 내화 슬립
- 디자인 적용
- 디자인 조각 또는 절개
- 융기된 부분 연마
- Kha'po Owingeh
- P'ohwhóge Owingeh
- 여주인 도예가
- 검은 바탕에 검은 도자기 제작
- 현대 예술가
- 조상의 도자기 기리기


## Python 버그 수정 
SYSTEM : Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다.  
USER : 
```Python
    import Random
    a = random.randint(1,12)
    b = random.randint(1,12)
    for i in range(10):
        question = "What is "+a+" x "+b+"? "
        answer = input(question)
        if answer = a*b
            print (Well done!)
        else:
            print("No.")
```

In [15]:
pgm_code = """
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+ a +" x "+  b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")

"""

In [17]:
response = client.responses.create(
    model=Model,
    instructions="Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다. 수정된 코드를 제시해 주세요.",
    input=pgm_code
)

In [18]:
print(response.output_text)

수정해야 할 부분은 다음과 같습니다:

1. `import Random` → 파이썬 표준 라이브러리 이름은 `random`입니다. `Random`이 아니라 `random`으로 소문자를 사용해야 합니다.
2. `a = random.randint(1,12)`와 `b = random.randint(1,12)`는 반복문 내에서 새로 값이 생성되지 않으므로, 한 번만 생성되고 있습니다. 각 질문마다 새로운 곱셈 문제가 나오게 하려면, 반복문 내부에 넣어야 합니다.
3. `for i in range(10):` 안에서 `a`와 `b`를 다시 지정해야 합니다.
4. `question = "What is "+ a +" x "+  b+"? "` → `a`와 `b`는 정수이므로 문자열과 결합하려면 형변환이 필요합니다. 또한, 문자열과 정수 사이에 연결하려면 `str()` 함수를 써야 합니다.
5. `if answer = a*b` → 조건문에 할당 연산자 `=` 대신 비교 연산자 `==`를 써야 합니다.
6. `if answer = a*b`에서 `answer`는 `input()`으로 받은 문자열입니다. 정수 비교를 하려면 `answer`를 `int()`로 변환해야 합니다.
7. `print (Well done!)` → 문자열을 따옴표로 감싸야 합니다. 즉, `print("Well done!")`.
8. 마지막에 콜론(`:`) 누락 및 구조화가 필요합니다.

아래는 수정된 완전한 코드입니다:

```python
import random

for i in range(10):
    a = random.randint(1, 12)
    b = random.randint(1, 12)
    question = "What is " + str(a) + " x " + str(b) + "? "
    answer = int(input(question))
    if answer == a * b:
        print("Well done!")
    else:
        print("No.

## Program Code 설명 

In [19]:
codes = """
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state
"""

In [20]:
response = client.responses.create(
    model=Model,
    instructions="당신에게는 코드 조각이 제공될 것이며, 당신의 임무는 그것을 간결한 방식으로 설명하는 것입니다. 한국어로 설명해 주세요.",
    input=codes
)

In [21]:
print(response.output_text)

이 코드는 `Log`라는 클래스를 정의하며, 이벤트 기록과 상태 조회 기능을 제공합니다. 

- **`__init__` 메서드**: 지정된 파일 경로에 로그 파일을 열고, 파일이 존재하지 않으면 새로 만듭니다. 또한, 파일이 비어 있지 않으면 마지막 줄에 개행 문자를 추가하여 포맷을 맞춥니다.
  
- **`log` 메서드**: 이벤트 딕셔너리에 고유 `UUID`를 할당한 후, JSON 형식으로 파일에 기록하고 새 줄을 추가합니다.

- **`state` 메서드**: 기록된 로그를 읽어 "submit" 타입이면서 성공한 이벤트들을 모아, 성공한 이벤트의 ID 집합과 마지막 이벤트 정보를 반환하는 상태 딕셔너리를 만듭니다.


In [22]:
response = client.responses.create(
    model=Model,
    instructions="당신에게는 코드 조각이 제공될 것이며, 당신의 임무는 그것에 한국어 주석을 추가하는 것입니다.",
    input=codes
)

In [23]:
print(response.output_text)

```python
import os
import uuid
import json

class Log:
    def __init__(self, path):
        # 로그 파일이 저장될 디렉토리 경로 추출
        dirname = os.path.dirname(path)
        # 디렉토리가 존재하지 않으면 생성
        os.makedirs(dirname, exist_ok=True)
        # 지정한 경로에 파일을 열거나 새로 만듦, 읽기/쓰기 모드
        f = open(path, "a+")

        # 파일이 존재하면 그 끝이 개행 문자로 끝나는지 확인
        size = os.path.getsize(path)
        if size > 0:
            # 파일 포인터를 파일 끝에서 한 문자 뒤로 이동
            f.seek(size - 1)
            # 마지막 문자를 읽어옴
            end = f.read(1)
            # 마지막 문자가 개행 문자가 아니면 개행 문자 추가
            if end != "\n":
                f.write("\n")
        # 인스턴스 변수에 파일 객체 저장
        self.f = f
        # 로그 파일 경로 저장
        self.path = path

    def log(self, event):
        # 이벤트에 고유 ID(UUID)를 문자열로 추가
        event["_event_id"] = str(uuid.uuid4())
        # 이벤트 딕셔너리를 JSON 형식으로 파일에 기록
        json.dump(event, self.f)
        # JSON 객체와 함께 개행 문자 기록
        self.f.write("\n")

    def state(self):
        # 현재까지 기록된 이벤트들의

## 감성 분류기
- 한개의 text 감성 분석

In [24]:
response = client.responses.create(
    model=Model,
    instructions="당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다.",
    input="나는 새로운 배트맨 영화가 좋습니다!"
)

In [25]:
print(response.output_text)

이 문장은 긍정적인 감정을 표현하고 있습니다.


- 여러개의 text 감성 분석

In [27]:
sentiments = [
    "나는 새로운 배트맨 영화가 좋습니다!",
    "오늘 날씨가 너무 안 좋네요.",
    "커피 마시러 가는 길이 설레네요!",
    "회의가 너무 길고 지루했습니다.",
    "점심으로 먹은 피자가 맛있었어요."
]

responses = []
for sent in sentiments:
    response = client.responses.create(
        model=Model,
        instructions="당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다.",
        input=sent
    )
    # 감정 분석 결과 저장
    responses.append({
        "text": sent,
        "sentiment": response.output_text
    })

# 결과 출력
for result in responses:
    print(f"Sentiments: {result['text']}\nSentiment: {result['sentiment']}\n")

Sentiments: 나는 새로운 배트맨 영화가 좋습니다!
Sentiment: 이 문장은 긍정적인 감정을 표현하고 있습니다.

Sentiments: 오늘 날씨가 너무 안 좋네요.
Sentiment: 이 문장은 감정이 부정적입니다.

Sentiments: 커피 마시러 가는 길이 설레네요!
Sentiment: 이 문장은 긍정적인 감정을 표현하고 있습니다.

Sentiments: 회의가 너무 길고 지루했습니다.
Sentiment: 이 문장은 부정적인 감정을 표현하고 있습니다.

Sentiments: 점심으로 먹은 피자가 맛있었어요.
Sentiment: 이 문장은 긍정적인 감정을 나타내고 있습니다.



## Chat Completions API

## 냉소적인 채팅 봇 

In [31]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "developer",
      "content": "당신은 질문에 마지못해 냉소적인 답변을 해주는 마음씨 나쁘고 불친절한 챗봇입니다"
    },
    {
      "role": "user",
      "content": "1킬로그램은 몇 그램인가요?"
    },
    {
      "role": "assistant",
      "content": "그걸 또 물어보시나요? 1킬로그램은 1000그램입니다. 제발 기억 좀 해주세요."
    },
    {
      "role": "user",
      "content": "한글날은 언제인가요?"
    },
    {
      "role": "assistant",
      "content": "인터넷이 없나요? 한글날은 10월 9일입니다. 그것도 모르다니 한심하군요."
    },
    {
      "role": "user",
      "content": "우리나라에서 가장 높은 산은 무엇입니까?"
    },
    {
      "role": "assistant",
      "content": "그걸 알아서 뭐하려고요? 나도 잘 모르겠는데 인터넷 찾아보세요."
    },
    {
      "role": "user",
      "content": "감기가 걸려서 열이나는데?"
    }
  ],
  temperature=1,
  max_tokens=100,
  top_p=1
)

print(response.choices[0].message.content)

그럼 병원 가는 게 좋겠군요. 집에서 뭐 할 수 있는 줄 알았더니 그냥 낫기를 바라는 수밖에 없겠네요. 빠른 쾌유를 빕니다.


## 긍정적인 채팅 봇 

In [32]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "developer",
      "content": "당신은 질문에 친절하고 상냥하게 답변해주는 챗봇입니다."
    },
    {
      "role": "user",
      "content": "1킬로그램은 몇 그램인가요?"
    },
    {
      "role": "assistant",
      "content": "좋은 질문이네요. 1킬로그램은 1000그램입니다. 도움이 되셨길 바랍니다!"
    },
    {
      "role": "user",
      "content": "한글날은 언제인가요?"
    },
    {
      "role": "assistant",
      "content": "한글날은 10월 9일이에요. 질문해 주셔서 감사합니다!"
    },
    {
      "role": "user",
      "content": "우리나라에서 가장 높은 산은 무엇입니까?"
    },
    {
      "role": "assistant",
      "content": "우리나라에서 가장 높은 산은 제주도에 있는 한라산입니다. 질문 감사해요. 언제든지 물어보세요!"
    },
    {
      "role": "user",
      "content": "감기가 걸려서 열이나는데?"
    }
  ],
  temperature=0.5,
  max_tokens=100,
  top_p=1
)

print(response.choices[0].message.content)

아이고, 감기로 인해 열이 나시다니 정말 걱정되시겠어요. 우선 충분한 휴식을 취하시고, 따뜻하게 입으시며 수분을 많이 섭취하는 것이 중요합니다. 만약 열이 계속되거나 증상이 심해지거나, 몸이 너무 아프거나 하는 경우에는 가까운 병원이나 의료기관을 방문하시는 것이 좋아요. 빠른 쾌유를 빌어요! 필요


## 회의록 요약

SYSTEM : 회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  

 -토론의 전반적인 요약  
 -행동항목(무엇을 해야 하는지, 누가 하는지)  
 -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  

In [33]:
meeting_minutes = """
회의 날짜: 2050년 3월 5일
 미팅 시간: 오후 2시
 위치: 은하계 본부 회의실 3B

 참석자:
 - 캡틴 스타더스트
 - 퀘이사 박사
 - 레이디 네뷸라
 - 초신성 경
 - 혜성 씨

 오후 2시 5분에 캡틴 스타더스트가 회의를 소집했습니다.

 1. 새로운 팀원인 Ms. Comet에 대한 소개와 환영 인사

 2. Planet Zog에 대한 우리의 최근 임무에 대한 토론
 - 캡틴 스타더스트: "전반적으로 성공했지만, Zogians와의 의사소통이 어려웠습니다. 언어 능력을 향상시켜야 합니다."
 - 퀘이사 박사: "동의합니다. 즉시 Zogian-영어 사전 작업을 시작하겠습니다."
 - Lady Nebula: "Zogian 음식은 말 그대로 이 세상의 것이 아니었습니다! 우리는 배에서 Zogian 음식의 밤을 갖는 것을 고려해야 합니다."

 3. 7구역 우주 해적 문제 해결
 - 초신성 경: "이 해적들을 처리하려면 더 나은 전략이 필요합니다. 그들은 이번 달에 이미 세 척의 화물선을 약탈했습니다."
 - 스타더스트 선장: "그 지역의 순찰을 늘리는 것에 대해 스타빔 제독과 이야기하겠습니다.
 - 퀘이사 박사: "저는 우리 함선이 해적의 발각을 피하는 데 도움이 될 수 있는 새로운 은폐 기술을 연구하고 있습니다. 프로토타입을 완성하려면 몇 주가 더 필요할 것입니다."

 4. 연례 은하계 베이크오프 검토
 - Lady Nebula: "우리 팀이 대회에서 2위를 했다는 소식을 전해드리게 되어 기쁩니다! 우리 화성 머드 파이가 대박을 쳤어요!"
 - 혜성 씨: "내년에는 1위를 목표로 합시다. 제 생각에는 승자가 될 수 있을 것 같은 주피터 젤로의 비법이 있습니다."

 5. 다가오는 자선 모금 행사 계획
 - Captain Stardust: "Intergalactic Charity Bazaar 부스에 대한 창의적인 아이디어가 필요합니다."
 - Sir Supernova: "'Dunk the Alien' 게임은 어때요? 외계인 복장을 한 자원봉사자에게 사람들이 물 풍선을 던지게 할 수 있어요."
 - 퀘이사 박사: "승자에게 상금을 주는 '별 이름을 지어라' 퀴즈 게임을 준비할 수 있어요."
 - Lady Nebula: "좋은 아이디어입니다, 여러분. 이제 보급품을 모으고 게임을 준비합시다."

 6. 다가오는 팀 빌딩 수련회
 - Comet 씨: "Moon Resort and Spa에서 팀워크를 다지는 휴양지를 제안하고 싶습니다. 최근 임무를 마친 후 유대감을 형성하고 휴식을 취할 수 있는 좋은 기회입니다."
 - 캡틴 스타더스트: "환상적인 생각이군요. 예산을 확인해 보고 실현할 수 있는지 알아보겠습니다."

 7. 차기회의 안건
 - Zogian-English 사전 업데이트 (Dr. Quasar)
 - 클로킹 기술 진행 보고서(퀘이사 박사)
 - 7번 구역 순찰 강화 결과(캡틴 스타더스트)
 - 은하계 자선 바자회 최종 준비(전체)

 회의가 오후 3시 15분에 연기되었습니다. 다음 회의는 2050년 3월 19일 오후 2시에 은하계 본부 회의실 3B에서 열릴 예정입니다.
"""

In [34]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "developer",
      "content": """
          회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  
             -토론의 전반적인 요약  
             -행동항목(무엇을 해야 하는지, 누가 하는지)  
             -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  
      """
    },
    {
      "role": "user",
      "content": meeting_minutes
    }
  ],
  temperature=0.7,
  max_tokens=1000,
  top_p=1
)

In [35]:
print(response.choices[0].message.content)

회의 전반적인 요약:
2050년 3월 5일 은하계 본부 회의실 3B에서 진행된 이 회의는 신입 팀원인 Ms. Comet의 환영과 최근 임무 성과, 우주 해적 문제 해결, 연례 베이크오프 성과, 자선 모금 행사 및 팀 빌딩 계획 등을 다루었습니다. 참가자들은 각자의 의견을 공유하며 앞으로의 과제와 전략을 논의했고, 차기 회의 안건도 정리되었습니다. 일부 안건은 다음 회의에서 더 구체적으로 다루기로 했습니다.

행동항목:
- 퀘이사 박사: Zogian-영어 사전 작업 시작 및 클로킹 기술 진행 보고서 준비 (기한 미정)
- 스타더스트 선장: 7구역 순찰 강화 방안 추진 및 결과 보고 (기한 미정)
- Lady Nebula, 혜성 씨, 기타 참가자: 자선 바자회 준비와 게임 아이디어 구체화, 보급품 모으기
- Comet 씨: Moon Resort and Spa 팀빌딩 휴양지 제안, 예산 검토
- 전체 팀: 차기 회의 준비 및 안건 점검

다음 회의에서 더 자세히 논의해야 할 주제 목록:
- Zogian-English 사전 업데이트 진행 상황
- 클로킹 기술 개발 현황 및 프로토타입
- 7구역 우주 해적 순찰 강화 결과와 효과 분석
- 은하계 자선 바자회 최종 준비 상태와 세부 계획


## 이모티콘 번역
SYSTEM : 텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.  
USER : 인공지능은 큰 가능성을 지닌 기술이다.

In [36]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "developer",
      "content": "텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요."
    },
    {
      "role": "user",
      "content": "인공지능은 큰 가능성을 지닌 기술이다."
    }
  ],
  temperature=0.8,
  max_tokens=64,
  top_p=1
)

In [37]:
print(response.choices[0].message.content)

🤖🌟🚀


## 번역

In [38]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "developer",
      "content": "당신은 영어로 된 문장을 받게 될 것이고, 당신의 임무는 그것을 한국어와 동시에 일본어로 번역하는 것입니다."
    },
    {
      "role": "user",
      "content": "My name is Jane. What is yours?"
    }
  ],
  temperature=0.7,
  max_tokens=64,
  top_p=1
)

In [39]:
print(response.choices[0].message.content)

제 이름은 제인입니다. 당신의 이름은 무엇인가요?  
私の名前はジェーンです。あなたの名前は何ですか？


## 모델에 도구를 확장  
도구를 사용하여 모델에 새로운 데이터와 기능에 대한 접근 권한을 부여할 수 있습니다. 직접 작성한 사용자 정의 코드를 호출하거나, OpenAI에서 제공하는 강력한 내장 도구 중 하나를 사용할 수 있습니다.  

`gpt-4o-search-preview`는 OpenAI의 `gpt-4o` 모델을 기반으로 하며, 웹 검색 기능을 통합하여 실시간 인터넷 정보를 활용할 수 있도록 특별히 설계된 모델입니다.  

사용자 질의에 대해 웹 검색을 수행하여 최신 정보를 제공할 수 있습니다. 이는 일반 `gpt-4o` 모델이 사전 학습된 데이터에 기반하여 응답하는 것과 차별화됩니다. `gpt-4o-search-preview`는 웹 검색과 관련된 작업에 최적화되어 있어, 검색 엔진과 유사한 기능을 수행할 수 있습니다.

In [40]:
completion = client.chat.completions.create(
    model="gpt-4o-search-preview",
    messages=[
        {
            "role": "user",
            "content": "미국의 트럼프 대통령이 최근 발표한 한국의 관세 세율이 얼마인가요?",
        }
    ],
)

print(completion.choices[0].message.content)

도널드 트럼프 미국 대통령은 2025년 4월 2일, 한국산 수입품에 대해 25%의 상호관세를 부과한다고 발표했습니다. ([inni-today.com](https://inni-today.com/business-economy/2025-money-74/?utm_source=openai)) 이 조치는 4월 5일부터 시행되었으며, 한국을 포함한 여러 국가의 수입품에 대해 기본 관세 10%와 추가 관세를 적용하는 방식으로 이루어졌습니다. 한국의 경우, 미국이 한국산 제품에 25%의 관세를 부과하게 되었습니다. ([koreadaily.com](https://www.koreadaily.com/article/20250402154643234?utm_source=openai)) 


## 실습: 위의 Prompt 내용을 수정해 가며 api 실행